In [1]:
import os
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torchvision.io import read_image

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# getting base path
PATH = os.getcwd() + '/gdrive/MyDrive/AIMIA/'

In [4]:
!pip install validators

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=a304c001d74bfaa9fc93ee8a981198e7bcd15bc2bca01f6f600636b11d9bfddb
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [5]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:14: UserWarning: p

In [6]:
# train_data_m contain all the images name and ids
all_data = pd.read_csv(PATH + 'train_data_m.csv')

image_ids = all_data['file_name'].to_numpy()
labels = all_data['target'].to_numpy()


X_train, X_test, y_train, y_test = train_test_split(image_ids, labels, test_size= 0.20, random_state=42)

# creating dataset for train_validation
train_df = pd.concat([pd.Series(X_train),pd.Series(y_train)],axis = 1)
train_df.to_csv(PATH +'train_data.csv',index = False)

# creating dataset for testing 
test_df = pd.concat([pd.Series(X_test),pd.Series(y_test)],axis = 1)
test_df.to_csv(PATH + 'test_data.csv',index = False)

In [7]:
class PneumothoraxImgDataset(Dataset):


    def __init__(self, annotations_file, img_dir, dim = 256):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform= transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((dim,dim)),
            transforms.ToTensor()
        ])
        

    def __len__(self):
        return len(self.img_labels)
    
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
Test_Dataset = PneumothoraxImgDataset(PATH + 'test_data.csv', PATH + 'small_train_data_set')

In [9]:
class TV_Dataset(Dataset):
    def __init__(self, file_names, labels, img_dir,dim = 256):
        self.img_name =file_names
        self.labels = labels
        self.img_dir = img_dir
        self.transform= transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((dim,dim)),
            transforms.ToTensor()
        ])
        
    def __len__(self):
        return len(self.img_name)
    
    def __getitem__(self,idx):
        img_path = os.path.join(self.img_dir, self.img_name[idx])
        image = read_image(img_path)
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        
        return image, label 

In [10]:
def train_val_dataset(train_path,path_dir):
    
    all_data = pd.read_csv(train_path)
    
    image_ids = all_data['0'].to_numpy()
    labels = all_data['1'].to_numpy()
    
    X_train, X_val, y_train, y_val = train_test_split(image_ids, labels, test_size= 0.20, random_state = 122)
    
    train_dataset = TV_Dataset(X_train, y_train, path_dir)
    val_dataset = TV_Dataset(X_val, y_val, path_dir)
    
    return train_dataset, val_dataset    

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
if device =='cuda':
  print(torch.cuda.memory_summary(device=device, abbreviated=False))

In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.classes = 2
        self.efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=False)
        self.efficientnet.stem.conv = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        self.efficientnet.classifier.fc = nn.Linear(1280, self.classes, bias = True)
        
    
    def forward(self,x):
        return self.efficientnet(x)
    
    

In [49]:
def train(model, criterion, optimizer, num_of_epochs):
    
    train_losses = []
    train_acc = []
    val_losses = []
    val_accuracies = []
    
    for _, epoch in tqdm(enumerate(range(num_of_epochs))):
        print(f'\nEpoch {epoch + 1}/{num_of_epochs}')
        
        model.train()
        
        running_loss = 0.
        running_accuracy = 0.
        
        
        train_dataset, val_dataset = train_val_dataset(PATH + 'train_data.csv', PATH + 'small_train_data_set')
        
        train_loader = DataLoader(train_dataset,batch_size=64)
        val_loader = DataLoader(val_dataset,batch_size=32)
        
        print('-----------Trainning in Progress --------------')
        for idx, data in tqdm(enumerate(train_loader),total = len(train_loader), position=0, leave=True):
            images, labels = data
            images = images.type(torch.float32).to(device)
            optimizer.zero_grad()
            
            outputs = model(images)
            labels = labels.type(torch.LongTensor).to(device)
            
            _ , preds = torch.max(outputs, 1)
            
            loss = criterion(outputs,labels)
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()*images.size(0)
            running_accuracy += torch.sum(preds == labels.data)

        epoch_loss = running_loss/len(train_dataset)
        epoch_accuracy = running_accuracy/len(train_dataset)

        train_losses.append(epoch_loss)
        train_acc.append(epoch_accuracy)
        
        print(f'Training Loss: {epoch_loss:.6f} Training Acc.: {epoch_accuracy:.6f}')
        
        model.eval()

        running_loss = 0
        running_accuracy = 0
        
        print('-----------Validation in Progress --------------')

        for idx, data in tqdm(enumerate(val_loader),total = len(val_loader), position=0, leave=True):
            images, labels = data
            images = images.type(torch.float32).to(device)
            
            outputs = model(images)
            labels = labels.type(torch.LongTensor).to(device)
            
            loss = criterion(outputs,labels)
            
            _ , preds = torch.max(outputs, 1)
            
            running_loss += loss.item()*images.size(0)
            running_accuracy += torch.sum(preds == labels.data)
        
        val_loss = running_loss/len(val_dataset)
        val_accuracy = running_accuracy/len(val_dataset)

        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        print(f'\nVal Loss: {val_loss:.4f} Val Acc.: {val_accuracy:.4f}\n')

    

    return  model, train_acc, train_losses,  val_losses, val_accuracies

In [50]:
def plot_matrics(train_acc, train_loss, val_loss, val_acc):
  """plotting accuracies and losses in iteration"""

  sns.set_theme()
  
  # creating data for x axes
  epochs = len(train_acc)
  epoch_ids = [epoch for epoch in range(1, epochs+1)]

  # dividing in sub-plots
  plt.figure()
  fig, axes = plt.subplots(1, 2, sharex=True, figsize=(15,8))

  # plot 1
  sns.lineplot(ax=axes[0], x=epoch_ids, y=train_acc)
  sns.lineplot(ax=axes[0], x=epoch_ids, y=val_acc)
  axes[0].set_title('Model Accuracy')
  axes[0].legend(['Train Acc', 'Val Acc'])

  # plot2 
  sns.lineplot(ax=axes[1], x=epoch_ids, y=train_loss)
  sns.lineplot(ax=axes[1], x=epoch_ids, y=val_loss)
  axes[1].set_title('Model Losses')
  axes[1].legend(['Train Loss', 'Val Loss'])

  plt.show()

  return None

In [51]:
def test(model, criterion):

  test_loader = DataLoader(Test_Dataset, batch_size=32)

  model.eval()

  running_loss = 0
  running_accuracy = 0

  print('-------Testing Model------------')
  for idx, data in tqdm(enumerate(test_loader),total = len(test_loader), position=0, leave=True):
    
    images, labels = data
    images = images.to(device)
    
    outputs = model(images)
    labels = labels.type(torch.LongTensor).to(device)
    
    loss = criterion(outputs,labels)
    
    _ , preds = torch.max(outputs, 1)
    
    running_loss += loss.item()*images.size(0)
    running_accuracy += torch.sum(preds == labels.data)
        
  test_loss = running_loss/len(Test_Dataset)
  test_accuracy = running_accuracy/len(Test_Dataset)


  print(f'\nTest Loss: {test_loss:.5f} Test Acc.: {test_accuracy:.5f}\n')

  return test_loss, test_accuracy

In [87]:
import logging
# setting up logger
logger = logging.getLogger('Model')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(PATH + 'model_logs.log', mode = "a")
fh.setLevel(logging.INFO)
logger.addHandler(fh)
# console output off
logger.propagate = False

In [96]:
def execute(version, model, criterion, optimizer, epochs, plotting = True, perform_testing = True):

  logger.info(f'Version: {version}\n')
  trianed_model, train_acc, train_loss, val_loss, val_acc = train(model, criterion, optimizer, num_of_epochs = epochs)

  logger.info('Epoch   Training Accuracy  Validation Accuracy')
  for idx in range(len(train_acc)):
    logger.info(f'{idx+1}      {train_acc[idx]:.5f}              {val_acc[idx]:.5f}')

  if plotting:
    print(type(train_acc[0]))
    plot_matrics(train_acc, train_loss, val_loss, val_acc)
    

  if perform_testing:
    test_loss, test_acc = test(trianed_model, criterion)
    logger.info(f'Testing Accuracy {test_acc:.5f}')

  return None

In [99]:
# model defination
model = NeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# args to model
args = dict()
args['version'] = 'v0'
args['model'] = model
args['criterion'] = criterion
args['optimizer'] = optimizer
args['epochs'] = 20
args['plotting'] = False
args['perform_testing'] = True

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [100]:
execute(**args)

0it [00:00, ?it/s]


Epoch 1/20
-----------Trainning in Progress --------------


100%|██████████| 21/21 [00:25<00:00,  1.20s/it]


Training Loss: 0.569929 Training Acc.: 0.765432
-----------Validation in Progress --------------


100%|██████████| 11/11 [00:05<00:00,  2.19it/s]
1it [00:30, 30.20s/it]

...

Training Loss: 0.000121 Training Acc.: 1.000000
-----------Validation in Progress --------------


100%|██████████| 11/11 [00:04<00:00,  2.20it/s]
20it [10:14, 30.70s/it]



Val Loss: 1.5744 Val Acc.: 0.7938

-------Testing Model------------


100%|██████████| 13/13 [04:09<00:00, 19.23s/it]


Test Loss: 1.84269 Test Acc.: 0.76847

